In [19]:
class SingleLDS_Gurobi(object):
    """LDS Estimator based on Gurobi
    """
    
    def __init__(self, **kwargs):
        super(SingleLDS_Gurobi, self).__init__()
        
    def load_heartrate(as_series=False):
        rslt = np.array([
            84.2697, 84.2697, 84.0619, 85.6542, 87.2093, 87.1246,
            86.8726, 86.7052, 87.5899, 89.1475, 89.8204, 89.8204,
            90.4375, 91.7605, 93.1081, 94.3291, 95.8003, 97.5119,
            98.7457, 98.904, 98.3437, 98.3075, 98.8313, 99.0789,
            98.8157, 98.2998, 97.7311, 97.6471, 97.7922])

        if as_series:
            return pd.Series(rslt)
        return rslt
    
    def estimate(self, X):
        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.

        Parameters
        ----------
        X: array
            Variable seen as input

        Returns
        -------
        X_predict: array
            regression predict values of X or residuals
        obj: num
            Objective value in optima
        """

        # Input data
        if len(np.transpose(X))!= 1:
            X = list(np.array(X).reshape(-1))
        N = len(X)
        print('N,X')
        
        # Create a new model
        e = gp.Env()
        e.setParam('TimeLimit', 1*10)
        m = gp.Model(env=e)
        
        # Create variables
        G = m.addVar(vtype='C', name="G")
        Fdash = m.addVar(vtype='C', name="Fdash")
        phi = m.addVars((N+1), name="phi", vtype='C')
        q = m.addVars(N, name="q", vtype='C')
        p = m.addVars(N, name="p", vtype='C')
        f = m.addVars(N, name="f", vtype='C')
        print("This model has",len(phi)+len(q)+len(p)+len(f)+2,"decision variables.")
        m.update() 

        # Set objective function
        obj = gp.quicksum((X[t]-f[t])*(X[t]-f[t]) for t in range(N))  
        obj += gp.quicksum(p[t]*p[t] for t in range(N)) 
        obj += gp.quicksum(q[t]*q[t] for t in range(N)) 

        m.setObjective(obj, GRB.MINIMIZE)

        #AddConstrs
        m.addConstrs((f[t] == Fdash*phi[t+1] + p[t]) for t in range(N))  
        m.addConstrs((phi[t+1] == G*phi[t] + q[t]) for t in range(N))  
        m.update()

        m.Params.NonConvex = 2
        
        #solve
        m.optimize()
        
        m.setParam(GRB.Param.OutputFlag, 0)
        print(f"m.status is " + str(m.status))
        print(f"GRB.Status.OPTIMAL is "+ str(GRB.Status.OPTIMAL))
        if m.status == GRB.Status.OPTIMAL:
            print(f"THIS IS OPTIMAL SOLUTION")
        else:
            print(f"THIS IS NOT OPTIMAL SOLUTION")
        print(f"Optimal objective value: {m.objVal}")
        print(f"Solution values: paras={G.X,Fdash.X}")
        
        
        data_dict = {'phi': [m.getAttr("x",phi)[h] for h in m.getAttr("x",phi)],
                     'p': [m.getAttr("x",p)[h] for h in m.getAttr("x",p)],
                     'q': [m.getAttr("x",q)[h] for h in m.getAttr("x",q)],
                     'f': [m.getAttr("x",f)[h] for h in m.getAttr("x",f)],
                     'X_Pred': [Fdash.X*m.getAttr("x",phi)[h+1]+m.getAttr("x",q)[h] for h in m.getAttr("x",f)]}
        df = pd.DataFrame.from_dict(data_dict, orient='index').transpose()
        
        '''def X_Pred(row):
               return Fdash.X*row["phi"]+row["q"]

        df["X_Pred"]=df.apply(lambda row:X_Pred(row),axis=1)'''
        f=m.getAttr("x",f)
        print("\nf:")

        print(df)


        return df
    
        #self.printSolution()
        
    def printSolution():
        if m.status == GRB.Status.OPTIMAL:
            print("\nError:%g" % m.objval)
            print("\nF:")
            F=m.getAttr("x",F)
            print("\nG:")
            G=m.getAttr("x",G)
            print("\nphi:")
            phi=m.getAttr("x",phi)
            print("\nq:")
            q=m.getAttr("x",q)
            print("\np:")
            p=m.getAttr("x",p)
        else:
            print("No solution")

        

In [20]:
class ClusterMultiLDS_Gurobi(object):
    """Clustering Estimator based on Gurobi
    """
    
    def __init__(self, **kwargs):
        super(ClusterMultiLDS_Gurobi, self).__init__()

    def estimate(self, X, K):
        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.

        Parameters
        ----------
        X: array
            Variable seen as input
        K: int
            Variable seen as number of clusters

        Returns
        -------
        X_predict: array
            regression predict values of X or residuals
        obj: num
            Objective value in optima
        """
        
        T = len(X)
        
        # Create a new model
        e = gp.Env()
        e.setParam('TimeLimit', 6*60)
        m = gp.Model(env=e)
        

        # Create indicator variable
        nL = len(np.transpose(X))
        X =[((l,t),np.transpose(X).iloc[l,t]) for l in range(nL) for t in range(T)]
        X = tupledict(X)

        #L = tupledict({(0,0):0, (0,1): 1, (0,2): 0, (0,3): 1, (0,4): 1, (0,5): 1})
        #N=T*nL
        #print('T is ' + str(T)+', Groups number is ' + str(nL))

        # Set hidden_state_dim(n)
        n0 = 2
        n1 = 4

        # Create variables
        L = m.addVars(1, nL, name="L", vtype='B')
        G0 = m.addVars(n0,n0, name="G0",vtype='C')
        phi0 = m.addVars(n0,(T+1), name="phi0", vtype='C')
        p0 = m.addVars(n0,T, name="p0", vtype='C')
        F0 = m.addVars(nL,n0, name="F0",vtype='C')
        q = m.addVars(nL,T, name="q", vtype='C')        
        f0 = m.addVars(nL,T, name="f0", vtype='C')
        f1 = m.addVars(nL,T, name="f1", vtype='C')
        G1 = m.addVars(n1,n1, name="G1",vtype='C')
        F1 = m.addVars(nL,n1, name="F1",vtype='C')
        phi1 = m.addVars(n1,(T+1), name="phi1", vtype='C')
        p1 = m.addVars(n1,T, name="p1", vtype='C')

        v0 = m.addVars(nL,T, name="v0", vtype='C')
        v1 = m.addVars(nL,T, name="v1", vtype='C')
        X0 = m.addVars(nL,T, name="X0", vtype='C')
        X1 = m.addVars(nL,T, name="X1", vtype='C')
        u = m.addVars(nL,nL, name="u", vtype='C')

        #model.addVars(2, 3)
        #model.addVars([0, 1, 2], ['m0', 'm1', 'm2'])
        print("This model has",n0*n0* 2+n0*nL +n0*(T+1)* 2+n0*T* 2+T*nL* 3+n1*n1* 2+n1*nL +n1*(T+1)* 2 +n1*T* 2,"decision variables.")

        obj = gp.quicksum(((X0[l,t]-f0[l,t])*(X0[l,t]-f0[l,t])+(X1[l,t]-f1[l,t])*(X1[l,t]-f1[l,t])) for l in range(nL) for t in range(T) )
        obj += gp.quicksum(0.0005*p0[n_,t]*p0[n_,t] for n_ in range(n0) for t in range(T)) 
        obj += gp.quicksum(0.0001*L[0,l]*u[l,l] for l in range(nL) for t in range(T)) 
        #obj += gp.quicksum(w1[l,l]*(1-L[0,l]) for l in range(nL) for t in range(T)) 
        obj += gp.quicksum(0.0005*p1[n_,t]*p1[n_,t] for n_ in range(n1) for t in range(T)) 
        obj += gp.quicksum(0.0001*(1-L[0,l])*u[l,l] for l in range(nL) for t in range(T)) 


        m.setObjective(obj, GRB.MINIMIZE)

        # AddConstrs
        #m.addConstrs((w0[l,l] == (X0[l,t]-f0[l,t])*(X0[l,t]-f0[l,t])) for l in range(nL) for t in range(T))
        #m.addConstrs((w1[l,l] == (X1[l,t]-f1[l,t])*(X1[l,t]-f1[l,t])) for l in range(nL) for t in range(T))
        m.addConstrs((u[l,l] == q[l,t]*q[l,t]) for l in range(nL) for t in range(T))
        m.addConstrs((phi0[n_,(t+1)] == G0[n_,n_]*phi0[n_,t] + p0[n_,t]) for n_ in range(n0) for t in range(T))               
        m.addConstrs((X0[l,t] == L[0,l] * X[l,t]) for l in range(nL) for t in range(T))
        m.addConstrs((v0[l,t] == F0[l,n_]*phi0[n_,(t+1)]) for l in range(nL) for n_ in range(n0) for t in range(T))   
        m.addConstrs((f0[l,t] == L[0,l] * v0[l,t] + L[0,l] * q[l,t]) for l in range(nL) for t in range(T))  
        m.addConstrs((phi1[n_,(t+1)] == G1[n_,n_]*phi1[n_,t] + p1[n_,t]) for n_ in range(n1) for t in range(T))  
        m.addConstrs((X1[l,t] == (1-L[0,l]) * X[l,t]) for l in range(nL) for t in range(T))
        m.addConstrs((v1[l,t] == F1[l,n_]*phi1[n_,(t+1)]) for l in range(nL) for n_ in range(n1) for t in range(T)) 
        m.addConstrs((f1[l,t] == (1-L[0,l])*v1[l,t] + (1-L[0,l])*q[l,t]) for l in range(nL) for t in range(T))  
        m.update()

        # Solve it!
        m.Params.NonConvex = 2
        #m.setParam('OutputFlag', 0)

        
        m.optimize()
        # if m.objVal <= 1:
        print(f"Optimal objective value: {m.objVal}")

        print(f"m.status is " + str(m.status))
        print(f"GRB.OPTIMAL is "+ str(GRB.OPTIMAL))

        if m.status == GRB.Status.OPTIMAL:
            print(f"THIS IS OPTIMAL SOLUTION")
        else:
            print(f"THIS IS NOT OPTIMAL SOLUTION")

        
        #if m.status == GRB.Status.OPTIMAL:
        #    print(f"THIS IS OPTIMAL SOLUTION")
        #    print(f"Optimal objective value: {m.objVal}")
            
        # Print solution
        print(f"Optimal L value:")
        print(np.array([m.getAttr("x",L)[h] for h in m.getAttr("x",L)]))  
        '''
        print(f"Optimal G0 value:")
        print(np.array([m.getAttr("x",G0)[h] for h in m.getAttr("x",G0)]).reshape(n, n))  
        print(f"Optimal G1 value:")
        print(np.array([m.getAttr("x",G1)[h] for h in m.getAttr("x",G1)]).reshape(n, n)) 
        print(f"Optimal F0 value:")
        print(np.array([m.getAttr("x",F0)[h] for h in m.getAttr("x",F0)]).reshape(N0, n))         
        print(f"Optimal F1 value:")
        print(np.array([m.getAttr("x",F1)[h] for h in m.getAttr("x",F1)]).reshape(N1, n))         
        print(f"Optimal phi0 value:")
        print(np.array([m.getAttr("x",phi0)[h] for h in m.getAttr("x",phi0)]).reshape(n, (T+1)))
        print(f"Optimal phi1 value:")
        print(np.array([m.getAttr("x",phi1)[h] for h in m.getAttr("x",phi1)]).reshape(n, (T+1)))
        print(f"Optimal f0 value:")
        print(np.array([m.getAttr("x",f0)[h] for h in m.getAttr("x",f0)]).reshape(N0, T)) 
        print(f"Optimal f1 value:")
        print(np.array([m.getAttr("x",f1)[h] for h in m.getAttr("x",f1)]).reshape(N1, T))         
    '''
        #return




# Test for Single_LDS

In [21]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import sys

test_data = SingleLDS_Gurobi().load_heartrate()
#print(test_data)
y_pred = SingleLDS_Gurobi().estimate(test_data)


N,X
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2428830
Academic license - for non-commercial use only - registered to hexiaoyu@fel.cvut.cz
Set parameter TimeLimit to value 10
This model has 119 decision variables.
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to hexiaoyu@fel.cvut.cz
Optimize a model with 0 rows, 119 columns and 0 nonzeros
Model fingerprint: 0x193f320a
Model has 87 quadratic objective terms
Model has 58 quadratic constraints
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+02, 2e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving a

# Test for ClusterMultiLDS

In [ ]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

Rawdata = pd.read_csv('HeartRate_MIT_Test.csv',sep=',',header='infer',nrows=8)
Y = pd.DataFrame(Rawdata.drop('Time',axis=1))
#print(Y)
# k cluster no.

y_pred = ClusterMultiLDS_Gurobi().estimate(Y,K=2)



Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2428830
Academic license - for non-commercial use only - registered to hexiaoyu@fel.cvut.cz
Set parameter TimeLimit to value 360
This model has 424 decision variables.
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to hexiaoyu@fel.cvut.cz
Optimize a model with 96 rows, 536 columns and 192 nonzeros
Model fingerprint: 0xc9ea93f9
Model has 336 quadratic objective terms
Model has 480 quadratic constraints
Variable types: 530 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  QObjective range [1e-03, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+